# MIoC — Step 3: Contact Extraction & QA (Shape vs Poly∩Metal)

This notebook detects contacts from the **original flat-field corrected** image using a **shape-based** method, then compares them with **anticipated contacts** derived from **poly** and **metal** overlays. Finally it emits a merged/validated contact set and a router-ready CPIN list.

**Pipeline**
1. Load flat-field-corrected die image (full-res).
2. **Shape-based** contact detection via Hough circles + edge-ring density test.
3. Load previously extracted **poly** and **metal** masks; derive anticipated contacts from **dilate(poly) ∩ metal**.
4. Compare sets (shape vs poly∩metal), highlight **matches** and **misses**.
5. Merge validated contacts → save CSV, overlay, and **CPIN** lines.

**Set your paths & parameters in the next cell, then run cells top→down.**

In [1]:
from pathlib import Path
import numpy as np
import cv2
import pandas as pd

# === USER CONFIG ===
DIE_IMAGE = Path('./images/ncr_8338d_greyscale_corrected.jpg')  # TODO: set your die image
POLY_MASK = Path('./out-lab-thin/mask_poly_clean.png')     # TODO: set to your step2 poly mask
METAL_MASK = Path('/out-lab-thin/mask_metal_thin.png')   # TODO: set to your step2 metal mask
OUTDIR = Path('./out-step3_contacts')                    # output directory

# Shape-based (contact) detection params
R_MIN, R_MAX = 8, 14          # px, expected radius band (10–12 typical)
HOUGH_DP = 1.2                 # Hough resolution ratio
HOUGH_MIN_DIST = 18            # px, min center distance
HOUGH_PARAM1 = 120             # Canny high threshold
HOUGH_PARAM2 = 20              # Hough accumulator threshold (lower → more circles)
MIN_EDGE_DENSITY = 0.12        # min edge density in ring to accept circle

# Anticipated contacts from poly∩metal
DILATE_POLY_PX = 2             # px, dilate poly before intersecting with metal
CC_MIN_AREA = 9                # px^2, minimum blob area to keep

# Matching tolerance between sets (distance in px)
MATCH_TOL = 6

OUTDIR.mkdir(parents=True, exist_ok=True)
print('Output →', OUTDIR)

Output → out-step3_contacts


In [2]:
# Utility helpers
def read_img(path: Path):
    img = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(f'Could not read image: {path}')
    return img

def circles_via_hough(gray, rmin, rmax, dp=1.2, min_dist=18, param1=120, param2=20):
    g = cv2.GaussianBlur(gray, (0,0), 1.0)
    circles = cv2.HoughCircles(g, cv2.HOUGH_GRADIENT, dp=dp, minDist=min_dist,
                               param1=param1, param2=param2,
                               minRadius=rmin, maxRadius=rmax)
    if circles is None:
        return []
    return [(float(x), float(y), float(r)) for x,y,r in circles[0]]

def annulus_edge_density(edge, cx, cy, r, band=2):
    H,W = edge.shape
    y, x = np.ogrid[:H, :W]
    dist = np.sqrt((x - cx)**2 + (y - cy)**2)
    mask = (dist >= (r - band)) & (dist <= (r + band))
    ring_area = np.count_nonzero(mask)
    if ring_area == 0:
        return 0.0
    hits = np.count_nonzero(edge & mask)
    return hits / float(ring_area)

def detect_contacts_shape(img_bgr, rmin=8, rmax=14, dp=1.2, min_dist=18, param1=120, param2=20,
                          min_edge_density=0.12, draw_radius_shrink=1):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, threshold1=param1//2, threshold2=param1)
    circles = circles_via_hough(gray, rmin, rmax, dp=dp, min_dist=min_dist, param1=param1, param2=param2)
    mask = np.zeros_like(gray, dtype=np.uint8)
    centers = []
    for (cx, cy, r) in circles:
        dens = annulus_edge_density(edges, cx, cy, r, band=2)
        if dens < min_edge_density:
            continue
        rr = max(int(round(r - draw_radius_shrink)), 1)
        cv2.circle(mask, (int(round(cx)), int(round(cy))), rr, 255, thickness=-1)
        centers.append({'x': int(round(cx)), 'y': int(round(cy)), 'r': float(r), 'edge_density': float(dens)})
    return mask, centers

def clean_mask(mask, open_ksize=3, close_ksize=3, min_area=9):
    m = mask.copy()
    if open_ksize and open_ksize>1:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (open_ksize, open_ksize))
        m = cv2.morphologyEx(m, cv2.MORPH_OPEN, k)
    if close_ksize and close_ksize>1:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (close_ksize, close_ksize))
        m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, k)
    num, lab, stats, cent = cv2.connectedComponentsWithStats((m>0).astype(np.uint8), 8)
    out = np.zeros_like(m)
    keep = []
    for i in range(1, num):
        area = stats[i, cv2.CC_STAT_AREA]
        if area >= min_area:
            out[lab==i] = 255
            cx, cy = cent[i]
            keep.append({'x': int(round(cx)), 'y': int(round(cy)), 'area': int(area)})
    return out, keep

def overlay_points(img_bgr, pts, color=(203,62,255), half=3):
    out = img_bgr.copy()
    for p in pts:
        x, y = int(p['x']), int(p['y'])
        cv2.rectangle(out, (x-half,y-half), (x+half,y+half), color, 1)
    return out

def anticipated_contacts_from_masks(poly_mask, metal_mask, dilate_px=2, cc_min_area=9):
    if dilate_px>0:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*dilate_px+1, 2*dilate_px+1))
        poly_d = cv2.dilate(poly_mask, k)
    else:
        poly_d = poly_mask
    inter = cv2.bitwise_and(poly_d, metal_mask)
    inter_clean, centers = clean_mask(inter, open_ksize=3, close_ksize=3, min_area=cc_min_area)
    return inter_clean, centers

def match_sets(A, B, tol=6):
    # Return (matches indices), (a_only idx), (b_only idx)
    a_only = []
    matches = []
    used_b = set()
    for i,a in enumerate(A):
        ax, ay = a['x'], a['y']
        bid = -1
        best = 1e9
        for j,b in enumerate(B):
            if j in used_b: continue
            d = (ax-b['x'])**2 + (ay-b['y'])**2
            if d < best:
                best = d; bid = j
        if bid>=0 and best <= tol*tol:
            matches.append((i,bid))
            used_b.add(bid)
        else:
            a_only.append(i)
    b_only = [j for j in range(len(B)) if j not in used_b]
    return matches, a_only, b_only

print('Helpers loaded.')

Helpers loaded.


## 1) Load images & detect shape-based contacts

In [ ]:
img = read_img(DIE_IMAGE)
shape_mask, shape_centers = detect_contacts_shape(
    img,
    rmin=R_MIN, rmax=R_MAX,
    dp=HOUGH_DP, min_dist=HOUGH_MIN_DIST,
    param1=HOUGH_PARAM1, param2=HOUGH_PARAM2,
    min_edge_density=MIN_EDGE_DENSITY
)
overlay_shape = overlay_points(img, shape_centers, color=(0,255,0))  # green
cv2.imwrite(str(OUTDIR/'shape_contact_mask.png'), shape_mask)
cv2.imwrite(str(OUTDIR/'overlay_shape_on_die.png'), overlay_shape)
pd.DataFrame(shape_centers).to_csv(OUTDIR/'contacts_shape.csv', index=False)
print(f"shape contacts: {len(shape_centers)}")

## 2) Load poly & metal overlays → anticipated contacts (dilate(poly) ∩ metal)

In [ ]:
poly_mask = cv2.imread(str(POLY_MASK), cv2.IMREAD_GRAYSCALE)
metal_mask = cv2.imread(str(METAL_MASK), cv2.IMREAD_GRAYSCALE)
if poly_mask is None or metal_mask is None:
    raise FileNotFoundError('Set POLY_MASK and METAL_MASK paths to your step2 outputs.')

pm_mask, pm_centers = anticipated_contacts_from_masks(poly_mask, metal_mask, dilate_px=DILATE_POLY_PX, cc_min_area=CC_MIN_AREA)
overlay_pm = overlay_points(img, pm_centers, color=(255,0,0))  # red
cv2.imwrite(str(OUTDIR/'pm_contact_mask.png'), pm_mask)
cv2.imwrite(str(OUTDIR/'overlay_pm_on_die.png'), overlay_pm)
pd.DataFrame(pm_centers).to_csv(OUTDIR/'contacts_pm.csv', index=False)
print(f"poly∩metal contacts: {len(pm_centers)}")

## 3) Compare sets (shape vs poly∩metal) and visualize misses

In [ ]:
matches, shape_only_idx, pm_only_idx = match_sets(shape_centers, pm_centers, tol=MATCH_TOL)
print('matches:', len(matches), 'shape_only:', len(shape_only_idx), 'pm_only:', len(pm_only_idx))

# Build comparison overlay
overlay_cmp = img.copy()
for i,j in matches:
    x,y = shape_centers[i]['x'], shape_centers[i]['y']
    cv2.rectangle(overlay_cmp, (x-3,y-3), (x+3,y+3), (255,255,255), 1)  # white for match
for i in shape_only_idx:
    x,y = shape_centers[i]['x'], shape_centers[i]['y']
    cv2.rectangle(overlay_cmp, (x-3,y-3), (x+3,y+3), (0,255,0), 1)      # green → shape-only
for j in pm_only_idx:
    x,y = pm_centers[j]['x'], pm_centers[j]['y']
    cv2.rectangle(overlay_cmp, (x-3,y-3), (x+3,y+3), (255,0,0), 1)      # red → pm-only

cv2.imwrite(str(OUTDIR/'overlay_compare_on_die.png'), overlay_cmp)

# Save a merged/contact_final list (union) and class labels
merged = []
used_pm = set(j for _,j in matches)
for i,_ in matches:
    m = dict(shape_centers[i])
    m['source'] = 'both'
    merged.append(m)
for i in shape_only_idx:
    m = dict(shape_centers[i]); m['source'] = 'shape'
    merged.append(m)
for j in pm_only_idx:
    m = dict(pm_centers[j]); m['source'] = 'pm'
    merged.append(m)

df_final = pd.DataFrame(merged)
df_final.to_csv(OUTDIR/'contacts_final.csv', index=False)
print('final contacts:', len(df_final))

## 4) Emit CPIN list for router and a standalone overlay

In [ ]:
cpin_path = OUTDIR/'contacts_as_cpin.txt'
with open(cpin_path, 'w') as f:
    for i, r in df_final.iterrows():
        f.write(f"PIN,CPIN,CONTACT_{i+1},{int(r['x'])},{int(r['y'])},NSEW,metal,owner=CONTACT,role=via\n")
print('Wrote', cpin_path)

overlay_final = overlay_points(img, df_final.to_dict('records'), color=(203,62,255))
cv2.imwrite(str(OUTDIR/'overlay_contacts_final.png'), overlay_final)
print('Saved final overlay image.')

## 5) Optional: grid snapping (if contacts sit on a lattice)
Enable and tune if you want to reduce subpixel jitter.


In [ ]:
ENABLE_SNAP = False      # set True to enable
DX, DY = 8, 8            # grid pitch in px
X0, Y0 = 0, 0            # grid origin in px

if ENABLE_SNAP:
    df_snap = df_final.copy()
    df_snap['x'] = ((df_snap['x'] - X0)/DX).round().astype(int)*DX + X0
    df_snap['y'] = ((df_snap['y'] - Y0)/DY).round().astype(int)*DY + Y0
    df_snap.to_csv(OUTDIR/'contacts_final_snapped.csv', index=False)
    with open(OUTDIR/'contacts_as_cpin_snapped.txt', 'w') as f:
        for i, r in df_snap.iterrows():
            f.write(f"PIN,CPIN,CONTACT_{i+1},{int(r['x'])},{int(r['y'])},NSEW,metal,owner=CONTACT,role=via\n")
    print('Wrote snapped contacts and CPIN file.')

### Notes
- **Green**: shape-only detections (might be true contacts missed by poly∩metal masks, or false positives—inspect).
- **Red**: poly∩metal-only detections (might be masked shapes, or mask errors—inspect).
- **White**: matched by both methods.
- Tweak **R_MIN/R_MAX**, **HOUGH_PARAM2**, and **MIN_EDGE_DENSITY** if over/under-detecting.
- Tweak **DILATE_POLY_PX** and **CC_MIN_AREA** for anticipated contacts.
- Final CPIN lines live in `contacts_as_cpin.txt` (and `_snapped` if enabled).